# Regresión logística

*¿Para qué se utiliza?*
La regresión logística se utiliza para modelar la probabilidad de ocurrencia de un evento binario, es decir, cuando la variable dependiente toma solo dos posibles valores (por ejemplo, 0 y 1, éxito o fracaso, compra o no compra). Es ampliamente empleada en contextos donde se desea predecir si algo sucederá o no, como en análisis de riesgo crediticio, diagnóstico médico, o comportamiento del consumidor.

*Variables consideradas*
- Variable dependiente: debe ser binaria (toma solo los valores 0 y 1).
- Variables independientes: pueden ser cuantitativas o categóricas, y se utilizan para explicar o predecir la probabilidad del evento de interés.

*¿Cómo funciona?*
A diferencia de la regresión lineal, en la regresión logística no se modela directamente la variable dependiente, sino la probabilidad de que esta tome el valor de 1. Para evitar problemas como obtener probabilidades fuera del intervalo [0, 1], se utiliza una transformación llamada logit, que modela el logaritmo de la razón de probabilidades:

ln⁡(p/(1-p))=β_0+β_1 X+...+β_q X_q

Aquí, p representa la probabilidad de que el evento ocurra. Esta transformación permite que el lado izquierdo de la ecuación tome cualquier valor real, mientras que el lado derecho es una función lineal de las variables independientes.


*Supuestos o requisitos principales*

Aunque la regresión logística no requiere normalidad de los errores ni homocedasticidad, sí se basa en algunos supuestos importantes:
- La relación entre las variables independientes y el logit de la probabilidad es lineal.
- Independencia de las observaciones.
- No debe haber multicolinealidad excesiva entre las variables independientes.
- Tamaño de muestra adecuado, especialmente cuando las clases están desbalanceadas.

*Evaluación de resultados*

Una vez estimado el modelo, se calcula la probabilidad estimada para cada observación. Si esta probabilidad es mayor que un umbral (usualmente 0.5), se clasifica la observación como un "1" (evento ocurre); de lo contrario, como un "0".
Adicionalmente, se pueden utilizar métricas como la razón de verosimilitud, pseudo R², curvas ROC, y matriz de confusión para evaluar el desempeño del modelo.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Crear un conjunto de datos sintético
data = pd.DataFrame({
    'Ingresos': [25, 45, 28, 35, 40, 60, 22, 38, 50, 48],
    'Edad':     [22, 35, 24, 32, 45, 52, 23, 41, 43, 36],
    'Compra':   [0, 1, 0, 0, 1, 1, 0, 1, 1, 1]  # Variable binaria dependiente
})

# Variables independientes (X) y variable dependiente (y)
X = data[['Ingresos', 'Edad']]
y = data['Compra']

# División en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelo de regresión logística
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# Predicción y evaluación
y_pred = modelo.predict(X_test)
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))


Observa que se realizó una división en conjunto de entrenamiento y prueba, esta división es fundamental para evaluar la capacidad de generalización del modelo. Si entrenamos y evaluamos sobre los mismos datos, podríamos obtener un modelo que se ajusta muy bien a esos datos específicos (sobreajuste), pero que no funciona bien con datos nuevos. Al separar un conjunto para prueba, podemos medir el rendimiento del modelo en datos no vistos.

También se puede implementar una validación cruzada. En la validación cruzada, en lugar de usar solo una partición entrenamiento-prueba, la validación cruzada divide los datos en varios subconjuntos (k-folds). El modelo se entrena en k-1 folds y se evalúa en el restante, repitiendo el proceso k veces. Esto proporciona una mejor estimación de la capacidad del modelo para generalizar a nuevos datos.


In [ ]:
# Evaluar precisión con validación cruzada de 5 pliegues
scores = cross_val_score(LogisticRegression(), X, y, cv=5, scoring='accuracy')
print(f'Precisión promedio (validación cruzada 5-fold): {np.mean(scores):.2f}')
print(f'Precisión por fold: {np.round(scores, 2)}')


La matriz de confusión te permite ver cuántas observaciones fueron correctamente clasificadas (verdaderos positivos y negativos) y cuántas no. Para obtenerla utilizamos cross_val_predict que entrena y evalúa el modelo como en la validación cruzada, pero en lugar de calcular solo una métrica, genera predicciones para cada observación como si fueran nuevas.

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Obtener predicciones con validación cruzada (5 pliegues)
y_pred_cv = cross_val_predict(LogisticRegression(), X, y, cv=5)

# Calcular la matriz de confusión
cm = confusion_matrix(y, y_pred_cv)

# Mostrar la matriz de confusión de forma visual
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap='Blues')  # Opcional: cambia el color
